In [0]:
from pyspark.sql.functions import expr
from sedona.register.geo_registrator import SedonaRegistrator
SedonaRegistrator.registerAll(spark)

/root/.ipykernel/1819/command-8347407064833988-1682010426:3: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)


True

In [0]:
column_names = ["postcode", "geom", "wkt"]
codepoint = spark.read.csv("/Volumes/alpha/default/os/code_point_open_white_horse.csv", header=False, sep="|").toDF(*column_names)

column_names = ["uprn", "easting", "northing", "lat", "lon", "geom", "wkt"]
uprn = spark.read.csv("/Volumes/alpha/default/os/open_uprn_white_horse.csv", header=False, sep="|").toDF(*column_names)

In [0]:
uprn = uprn.withColumn("geom", expr("ST_GeomFromWKT(wkt, 27700)"))
uprn.createOrReplaceTempView("uprn")

codepoint = codepoint.withColumn("geom", expr("ST_GeomFromWKT(wkt, 27700)"))
codepoint.createOrReplaceTempView("codepoint")

In [0]:
sql = """
    WITH knn AS (
        SELECT 
            A.uprn as origin,
            B.postcode as destination,
            round(ST_Distance(A.geom, B.geom),2) as distance,
            row_number() OVER (PARTITION BY A.uprn ORDER BY A.uprn,ST_Distance(A.geom, B.geom) ASC, B.postcode) AS rn,
            true as is_valid
        FROM
            uprn A,
            codepoint B
        WHERE
            ST_DWithin(A.geom, B.geom, 5000)
        ORDER BY
            A.uprn,
            ST_Distance(A.geom, B.geom) ASC,
            destination
    )
    SELECT origin, destination, distance FROM knn WHERE rn = 1      
"""
knn = spark.sql(sql)

In [0]:
knn.show()

+------------+-----------+--------+
|      origin|destination|distance|
+------------+-----------+--------+
| 10009424628|    SN6 8LG| 1009.31|
| 10009424630|   RG17 8RE| 1121.34|
| 10009424631|   RG17 8RE| 1205.33|
|100120897368|   OX14 3JD|   32.25|
|100120897369|   OX14 3JD|     0.0|
|100120897370|   OX14 3JD|    4.12|
|100120897371|   OX14 3JD|    9.06|
|100120897372|   OX14 3JD|   16.03|
|100120897374|   OX14 2EY|   57.28|
|100120897375|   OX14 2DY|   53.81|
|100120897376|   OX14 2EY|    49.2|
|100120897377|   OX14 2EY|   50.09|
|100120897378|   OX14 2DT|   36.25|
|100120897379|   OX14 2EY|   48.27|
|100120897380|   OX14 2DT|   35.36|
|100120897381|   OX14 2EY|    50.0|
|100120897382|   OX14 2DT|   39.85|
|100120897383|   OX14 2DT|   53.37|
|100120897384|   OX14 2DT|   46.96|
|100120897385|   OX14 2DX|   48.37|
+------------+-----------+--------+
only showing top 20 rows


In [0]:
# Save the result to the catalog
knn.write.mode("overwrite").saveAsTable("knn_results")
#save as csv
knn.coalesce(1).write.mode("overwrite").csv("/Volumes/alpha/default/os/knn_results.csv", header=True)